In [1]:
import pandas as pd
import glob
import datetime
import numpy as np
today_str=str(datetime.datetime.now().date())

In [2]:
folder_file_pattern=glob.glob("/home/jian/Projects/Big_Lots/Sales_All/Sales_Data/*.txt")

In [3]:
taxonomy=pd.read_csv("/home/jian/BigLots/static_files/MediaStorm Data Extract - Product Taxonomy.txt",sep="|",dtype=str)
depart_name=pd.read_csv("/home/jian/BigLots/static_files/MediaStorm Data Extract - Department Names.txt",sep="|",dtype=str)
class_name=pd.read_csv("/home/jian/BigLots/static_files/MediaStorm Data Extract - Class Names.txt",sep="|",dtype=str,encoding = "ISO-8859-1")

In [4]:
depart_class=taxonomy[['department_id','class_code_id']].drop_duplicates()
depart_class=pd.merge(depart_class,depart_name,on='department_id',how="left")
depart_class=pd.merge(depart_class,class_name,on='class_code_id',how="left")

In [5]:
depart_class.head(2)

,department_id,class_code_id,department_desc,class_code_desc
0,108,11099,CANDYSNACK,FINANCE USE ONLY
1,111,11199,DIRECT STORE DELIVERIES,FINANCE USE ONLY


In [6]:
sales_all=pd.DataFrame()
for file in folder_file_pattern:
    df=pd.read_csv(file,sep="|",dtype=str,na_values="?")
    if "class_code_id" in df.columns:


        df['class_gross_sales_amt']=df['class_gross_sales_amt'].astype(float)
        df['class_gross_sales_amt']=df['class_gross_sales_amt'].fillna(0.0)
        df=df[['location_id','week_end_dt','class_code_id','class_gross_sales_amt']].drop_duplicates()
        df.reset_index(inplace=True)
        try:
            df['week_end_dt']=df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
            
        except:
            print(file,"Date Error")
            
            
    else:
        print (file,"Error: class_code_id not in columns")
        
    sales_all=sales_all.append(df)

In [7]:
Q1_sale_2017=sales_all[(sales_all['week_end_dt']>=datetime.datetime(2017,2,11).date()) & (sales_all['week_end_dt']<=datetime.datetime(2017,5,6).date())]
Q1_sale_2018=sales_all[(sales_all['week_end_dt']>=datetime.datetime(2018,2,10).date()) & (sales_all['week_end_dt']<=datetime.datetime(2018,5,5).date())]

In [8]:
Q1_sales_all=Q1_sale_2017.append(Q1_sale_2018)
Q1_sales_all=Q1_sales_all.drop_duplicates()
Q1_sales_all=pd.merge(Q1_sales_all,depart_class,on="class_code_id",how="left")
del Q1_sales_all['index']

In [12]:
Inclusion_stores=pd.read_excel("/home/jian/Projects/Big_Lots/Q1_Post/BL_Sales YoY_JL_20180618.xlsx",dtype=str,sheetname="Inclusion Stores",skiprows=1)
Store_Inclusion_List=Inclusion_stores['location_id'].unique().tolist()
Inclusion_stores.head(2)


,location_id,DMA,Q1 Campaign Type,2017-02-11,2017-02-18,2017-02-25,2017-03-04,2017-03-11,2017-03-18,2017-03-25,...,2018-03-03.1,2018-03-10.1,2018-03-17.1,2018-03-24.1,2018-03-31.1,2018-04-07.1,2018-04-14.1,2018-04-21.1,2018-04-28.1,2018-05-05.1
0,1,"COLUMBUS, OH",TV+Digital,65529.31,72052.29,144591.9,111094.38,91900.32,92978.17,88751.56,...,2566,2453,2369,2393,2295,2377,2712,2097,2160,2352
1,3,AUGUSTA-AIKEN,Balance,59518.92,67264.36,124226.79,105042.77,85331.1,78149.36,80778.23,...,2480,2194,2070,2093,2186,2320,2180,1935,1934,2139


In [15]:
Store_DMA=Inclusion_stores[['location_id','DMA','Q1 Campaign Type']]

In [17]:
Q1_sales_all=pd.merge(Q1_sales_all,Store_DMA,on="location_id",how="left")

In [19]:
Q1_sales_all.head(2)

,location_id,week_end_dt,class_code_id,class_gross_sales_amt,department_id,department_desc,class_code_desc,DMA,Q1 Campaign Type
0,1,2017-02-11,11001,1027.22,108,CANDYSNACK,CANDY,"COLUMBUS, OH",TV+Digital
1,1,2017-02-11,11002,436.71,108,CANDYSNACK,SEASONAL CANDY,"COLUMBUS, OH",TV+Digital


In [39]:
Q1_sales_all_In=Q1_sales_all[Q1_sales_all['location_id'].isin(Store_Inclusion_List)]



In [55]:
test_check=Q1_sales_all_In[Q1_sales_all_In['class_code_id']=="21012"]

In [59]:
test_check.head(2)

,location_id,week_end_dt,class_code_id,class_gross_sales_amt,department_id,department_desc,class_code_desc,DMA,Q1 Campaign Type
65,1,2017-02-11,21012,317.97,210,LAWN & GARDEN,PATIO FURNITURE,"COLUMBUS, OH",TV+Digital
288,1,2017-02-18,21012,113.99,210,LAWN & GARDEN,PATIO FURNITURE,"COLUMBUS, OH",TV+Digital


In [56]:
test_check_2017=test_check[(test_check['week_end_dt']>=datetime.date(2017,1,1))&(test_check['week_end_dt']<=datetime.date(2018,1,1))]
test_check_2017['class_gross_sales_amt'].sum()

53635511.780000001

In [57]:
test_check_2018=test_check[test_check['week_end_dt']>=datetime.date(2018,1,1)]
test_check_2018['class_gross_sales_amt'].sum()

53965461.140000001

In [58]:
test_check['class_gross_sales_amt'].sum()

107600972.92

In [67]:
week_13=pd.read_csv("/home/jian/BigLots/MediaStorm 2018-04-14/MediaStormSalesWeekly.txt",dtype=str,sep="|")
week_13['class_gross_sales_amt']=week_13['class_gross_sales_amt'].astype(float)

week_13[week_13['class_code_id']=="21012"]['class_gross_sales_amt'].sum()

8466860.7600000016

In [66]:
Q1_sales_all[(Q1_sales_all['week_end_dt']==datetime.date(2018,4,14)) & (Q1_sales_all['class_code_id']=="21012")]['class_gross_sales_amt'].sum()

8466860.7600000016

In [62]:
Q1_sales_all_In.head(2)

,location_id,week_end_dt,class_code_id,class_gross_sales_amt,department_id,department_desc,class_code_desc,DMA,Q1 Campaign Type
0,1,2017-02-11,11001,1027.22,108,CANDYSNACK,CANDY,"COLUMBUS, OH",TV+Digital
1,1,2017-02-11,11002,436.71,108,CANDYSNACK,SEASONAL CANDY,"COLUMBUS, OH",TV+Digital


In [22]:
Q1_sales_all.to_csv("/home/jian/Projects/Big_Lots/Sales_All/BL_Q1 Product_"+today_str+".csv")